In [ ]:
from src.guitar.Guitar import Guitar
from src.hand.LeftFinger import LeftFinger
from src.hand.LeftHand import LeftHand
from src.guitar.GuitarString import createGuitarStrings
from src.midi.midiToNote import midiToGuitarNotes,processedNotes
from src.utils.utils import convertNotesToChord, convertChordTofingerPositions
from src.HandPoseRecorder import HandPoseRecorder,HandPoseRecordPool
import copy
import time
import itertools
import json

## Import Midi file 导入Midi文件
Import midi file and convert it to a list of dict which including notes and beat. 导入midi文件，并将其转换为一个列表，这个列表里的元素是包含音符和节拍信息的字典。

In [ ]:
midiFilePath = "asset/Corridors Of Time Fingerstyle.mid"
filename = midiFilePath.split("/")[-1].split(".")[0]

GuitarNotes = midiToGuitarNotes(midiFilePath,useChannel=1)

GuitarNotes

## init guitar and handPoseRecordPool 初始化吉他和handPoseRecordPool
init guitar with a list of string. here we use a standard 6-string guitar. 用一个字符串列表来初始化吉他。这里我们使用标准的6弦吉他。
init the start hand pose. here we put a hand at barrel 0 and put fingers on string 3, which is G string. 初始化起始手型。在这里，我们将设定初始手型位置为第0品，并将手指放在第3弦上，也就是G弦上。

handPoseRecordPool can init with param `size`, which is the size of the pool. Bigger size will cost more memory and more time but can calculate more possibilities. handPoseRecordPool可以通过参数`size`初始化，即池的大小。越大就会消耗更多的内存和时间，但可以计算更多可能性。

In [ ]:
# 设定各弦音高
guitar_string_list = createGuitarStrings(["e", "b", "G", "D", "A", "E1"])
# 初始化吉它
guitar = Guitar(guitar_string_list)
# 设定各手指状态
leftFingers = [
    LeftFinger(1, guitar_string_list[1], 1),
    LeftFinger(2, guitar_string_list[2], 2),
    LeftFinger(3, guitar_string_list[3], 3),
    LeftFinger(4, guitar_string_list[4], 4)
]
# 初始化左手
initLeftHand = LeftHand(leftFingers)
# 初始化第一个记录器
handPoseRecord = HandPoseRecorder()
handPoseRecord.addHandPose(initLeftHand, 0,0)
# 初始化记录池
handPoseRecordPool = HandPoseRecordPool(100)
handPoseRecordPool.append(handPoseRecord)

current_recoreder_num = 0
previous_recoreder_num = 0

## Calculate the possibility of hand pose 计算手型的可能性
- For each notes, we calculate all possibile chords, which includes position infomations of notes on guitar. 对于每个音符，我们计算所有可能的和弦，包括音符在吉他上的位置信息。
- For each chords, we calculate all possibile hand poses, which includes the position of fingers on guitar.
- For each hand poses, we generate a LeftHand object if it is possible to play. 对于每个和弦，我们计算所有可能的手型，包括手指在吉他上的位置。
- For each LeftHand object, we calculate entropy, the cost of switch hand pose. Then we append it to the handPoseRecord. 对于每个手型，如果可以演奏，我们就生成一个LeftHand对象。
- For each handPoseRecord, we put it in the handPoseRecordPool if its entropy is lower than the record in the pool. 对于每个LeftHand对象，我们计算熵，即切换手型的成本。然后将其添加到handPoseRecord中。
- After all notes are calculated, we get the best handPoseRecord from the pool. Then print the result. 计算完所有音符后，我们从池中获取最佳handPoseRecord，然后打印结果


In [ ]:
for i in range(len(GuitarNotes)):
    start_time = time.time()
    guitarNote = GuitarNotes[i]
    notes = guitarNote["notes"]
    real_time = guitarNote["real_time"]
    notes = processedNotes(notes)
    
    # calculate all possible chords, including the position information of notes on the guitar. 计算所有可能的和弦,包含音符在吉它上的位置信息。
    chords = convertNotesToChord(notes, guitar)
    
    # init current record list. 记录池先更新初始化当前记录列表。
    handPoseRecordPool.readyForRecord()
    
    fingerPositionsList = []
    handPoseRecordCount = 0
        
    # calculate all possible fingerings, including the position information of fingers on the guitar. 计算所有可能的按法，包含手指在吉它上的位置信息。
    for chord in chords:
        fingberPositions = convertChordTofingerPositions(chord) 
        if len(fingberPositions) == 0:
            continue       
        fingerPositionsList += fingberPositions
        
    if len(fingerPositionsList) == 0:
        print("当前时间是{real_time}，没有找到合适的按法。")  
    for handPoseRecord, fingerPositions in itertools.product(handPoseRecordPool.preHandPoseRecordPool, fingerPositionsList):
        oldhand = handPoseRecord.currentHandPose()
        # Iterate through the list of fingerings, generate a new LeftHand object based on the fingering. 遍历按法列表，根据按法生成新的LeftHand对象。
        hand, entropy = oldhand.generateNextHands(
            guitar, fingerPositions)
        if hand is not None:                        
            newHandPoseRecord = copy.deepcopy(handPoseRecord)
            newHandPoseRecord.addHandPose(hand, entropy, real_time)
            handPoseRecordPool.append(newHandPoseRecord)
            handPoseRecordCount += 1
    
    end_time = time.time()
    previous_recoreder_num = current_recoreder_num
    current_recoreder_num = len(handPoseRecordPool.curHandPoseRecordPool)
    if current_recoreder_num < previous_recoreder_num:
        print(f"当前record数量是{current_recoreder_num}，上一次record数量是{previous_recoreder_num}，这一轮操作一共append了{handPoseRecordCount}个record。")
    print(f"当前时间是{real_time}，当前有{len(handPoseRecordPool.curHandPoseRecordPool)}个rerocder，耗时{end_time-start_time:.2f}秒。")


# after all iterations, read the best solution in the record pool. 全部遍历完以后，读取记录池中的最优解。
bestHandPoseRecord = handPoseRecordPool.curHandPoseRecordPool[0]
bestEntropy = bestHandPoseRecord.currentEntropy
print(f"最小消耗熵为：{bestEntropy}")
bestHandPoseRecord.output(True)
bestHandPoseRecord.save(f"output/{filename}_lefthand_recorder.json")


## Initialize Animation Values 初始化动画所需数值

In Blender, by determining the positions of the following key points: IK pivot point, Y-axis rotation value, we can determine the entire left hand's posture. These key points are:
在blender中，通过确定以下几个关键点的位置，ik轴点，y轴旋转值，就可以确定整个左手的状态，这几个关键点分别是：
- Palm 手掌
- Thumb 大拇指
- Index finger 食指
- Middle finger 中指
- Ring finger 无名指
- Pinky finger 小指

There are four extreme positions of the hand shape, which are:
而手型有四个极端位置，分别是：
- Open string position on the 1st string 1弦空品
- Open string position on the 6th string 6弦空品
- 12th fret position on the 1st string 1弦12品
- 12th fret position on the 6th string 6弦12品

For each extreme position, we can adopt three methods to press the strings:
而每个极端位置，我们都可以采用三种方式来按弦，分别是：
- Index finger and middle finger on different frets, with the whole palm parallel to the neck 食指和中指在不同品上，整个手掌平行于琴颈
- Index finger and middle finger on the same fret, with the index finger pointing outward and the middle finger pointing inward, requiring the palm to rotate at a certain angle 食指与中指同品，而食指向外，中指向内，此时手掌要旋转一定角度
- Index finger and middle finger on the same fret, with the index finger pointing inward and the middle finger pointing outward, requiring the palm to rotate at a certain angle 食指与中指同品，而食指向内，中指向外，此时手掌要旋转一定角度

In Blender, we record the values of all key points for these four extreme positions in three different ways, totaling 12 hand shapes. 在blender中，我们记录下来以上四个极端位置的三种不同方式下，所有关键点的值，共计12种手型。

Through these 12 hand shapes, we can obtain the hand shape at any position through interpolation. 通过这12种手型，我们可以通过插值的方式，得到任意位置的手型。

In [1]:

# frames per second. blender文件中的动画帧率
FPS = 24

midiFilePath = "asset/Corridors Of Time Fingerstyle.mid"
filename = midiFilePath.split("/")[-1].split(".")[0]

recorder = f"output/{filename}_lefthand_recorder.json"
animation = f"output/{filename}_lefthand_animation.json"

In [2]:
from src.animate.animate import leftHand2Animation
leftHand2Animation(recorder, animation, FPS)

## 计算左手的动作

和计算右手的原理类似，我们的左手从一个初始手型开始，然后每一次需要拨弦时，就计算下一次可能有哪些拨弦手型，并且计算每个手型所消耗的熵，把这些手型都记录在一个rightHandRecorder里，最终反复迭代，选出消耗熵最小的拨弦序列。

In [ ]:
from src.hand.RightHand import RightHand
from src.HandPoseRecorder import RightHandRecorder,RightHandRecordPool
initRightHand = RightHand(usedFingers=[],rightFingerPositions=[5,2,1,0])

initRightHandRecorder = RightHandRecorder()
initRightHandRecorder.addHandPose(initRightHand, 0,0)

rightHandRecordPool = RightHandRecordPool(100)
rightHandRecordPool.append(initRightHandRecorder)

In [ ]:
rightHandRecordPool.curHandPoseRecordPool[0].handPoseList[0].output()

In [ ]:
import json
import time
import copy



from src.utils.utils import generatePossibleRightHands

midiFilePath = "asset/Corridors Of Time Fingerstyle.mid"

filename = midiFilePath.split("/")[-1].split(".")[0]

inputFile = f"output/{filename}.json"

rightHandRecorderFile = f"output/{filename}_righthand_recorder.json"


with open(inputFile, "r") as f:

    data = json.load(f)

    for item in data:
        counter = 0
        start_time = time.time()

        real_time = item["real_time"]

        leftHand = item["leftHand"]

        usedStrings = []

        for finger in leftHand:

            if finger["fingerIndex"] == -1 or finger["fingerInfo"]["press"] != 0:

                usedStrings.append(finger["fingerInfo"]["stringIndex"])
                
        if usedStrings == []:
            continue
        print(real_time)
        print(usedStrings)
        rightHandRecordPool.readyForRecord()

        allFingers = ["p","i","m","a"]

        allstrings = [0,1,2,3,4,5]

        possibleRightHands = generatePossibleRightHands(

            usedStrings, allFingers, allstrings)
        

        for rightHand in possibleRightHands:
            for handRecorder in rightHandRecordPool.preHandPoseRecordPool:
                newRecorder = copy.deepcopy(handRecorder)
                lastHand = newRecorder.currentHandPose()
                entropy = lastHand.caculateDiff(rightHand)
                newRecorder.addHandPose(rightHand, entropy, real_time)
                rightHandRecordPool.append(newRecorder)
            
        end_time = time.time()
        print(f"当前时间是{real_time}，当前有{len(rightHandRecordPool.curHandPoseRecordPool)}个rerocder，耗时{end_time-start_time:.2f}秒。")

In [ ]:
# after all iterations, read the best solution in the record pool. 全部遍历完以后，读取记录池中的最优解。
bestHandPoseRecord = rightHandRecordPool.curHandPoseRecordPool[0]
bestEntropy = bestHandPoseRecord.currentEntropy
print(f"最小消耗熵为：{bestEntropy}")
bestHandPoseRecord.output()
bestHandPoseRecord.save(rightHandRecorderFile)

In [3]:
FPS = 24

midiFilePath = "asset/Corridors Of Time Fingerstyle.mid"
filename = midiFilePath.split("/")[-1].split(".")[0]

recorder = f"output/{filename}_righthand_recorder.json"
rightHandAnimationFile = f"output/{filename}_righthand_animation.json"

In [4]:
from src.animate.animate import rightHand2Animation
rightHand2Animation(recorder, rightHandAnimationFile,FPS)